In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 16 16:39:52 2019

"""

import pandas as pd
import query_code as q
import data_process as process
import file_data_att as fd
from typing import Dict
import settings
from queries_PIM import _attr_query, sys2_attr_values, sys2_attr_query
import time


pd.options.mode.chained_assignment = None

ModuleNotFoundError: No module named 'query_code'

In [ ]:
def match_category(df):
    """compare data colected from matching file (match_df) with sys1 and sys2 data pulls and create a column to tell analysts
    whether attributes from the two systems have been matched"""

    for row in df.itertuples():
        if (row.Index, row.alt_sys1_name) == (row.Index, row.alt_sys2_name):
            df.at[row.Index,'Matching'] = 'Match'
        elif process.isBlank(row.sys1_Attribute_Name) == False:
            if process.isBlank(row.sys2_Attribute_Name) == True:
                df.at[row.Index,'Matching'] = 'sys1 only'
        elif process.isBlank(row.sys1_Attribute_Name) == True:
            if process.isBlank(row.sys2_Attribute_Name) == False:
                df.at[row.Index,'Matching'] = 'sys2 only'
             
    return df


def sys2_process(count, order, node, sys2_dict: Dict, k):
    """if sys2 node has not been previously processed (in sys2_dict), process and add it to the dictionary"""
    sys2_sample_vals = pd.DataFrame()
    sys2_att_vals = pd.DataFrame()

    sys2_df = q.sys2_atts(sys2_attr_query, node, 'tax.id')  #tprod."categoryId"')  #get sys2 attribute values for each sys2_l3 node\
    
    if sys2_df.empty==False:
        sys2_att_vals, sys2_sample_vals = q.sys2_values(sys2_attr_values, node, 'tax.id') #sys2_values exports a list of --all-- normalized values and sample_values
        
        if sys2_att_vals.empty==False:
            sys2_sample_vals = sys2_sample_vals.rename(columns={'Normalized Value': 'sys2 Attribute Sample Values'})
            sys2_df = pd.merge(sys2_df, sys2_sample_vals, on=['sys2_Attribute_Name'])  #add t0p 5 normalized values to report
            sys2_df = pd.merge(sys2_df, sys2_att_vals, on=['sys2_Attr_ID'])  #add t0p 5 normalized values to report
        else:
            if count == 1:
                order = 'alt'

        sys2_df = sys2_df.drop_duplicates(subset='sys2_Attr_ID')  #sys2 attribute IDs are unique, so no need to group by pim node before getting unique
        sys2_df['alt_sys2_name'] = process.process_att(sys2_df['sys2_Attribute_Name'])  #prep att name for merge
        
        sys2_dict[node] = sys2_df #store the processed df in dict for future reference
 
    else:
        print('{} EMPTY DATAFRAME'.format(node))    
        
    return sys2_dict, sys2_df, order


def sys1_assign_nodes (sys1_df, sys2_df):
    """assign sys2 node data to sys1 columns"""
    
    att_list = []
    
    node_ID = sys2_df['sys2_Node_ID']
    cat_ID = sys2_df['sys2_Category_ID']
    cat_name = sys2_df['sys2_Category_Name']
    node_name = sys2_df['sys2_Node_Name']
    pim_path = sys2_df['sys2_PIM_Path']

    att_list = sys1_df['sys1_Attribute_Name'].unique()
    
    for att in att_list:
        sys1_df.loc[sys1_df['sys1_Attribute_Name'] == att, 'sys2_Node_ID'] = node_ID
        sys1_df.loc[sys1_df['sys1_Attribute_Name'] == att, 'sys2_Category_ID'] = cat_ID
        sys1_df.loc[sys1_df['sys1_Attribute_Name'] == att, 'sys2_Category_Name'] = cat_name
        sys1_df.loc[sys1_df['sys1_Attribute_Name'] == att, 'sys2_Node_Name'] = node_name
        sys1_df.loc[sys1_df['sys1_Attribute_Name'] == att, 'sys2_PIM_Path'] = pim_path
    
    return sys1_df


def sys2_assign_nodes (sys1_df, sys2_df):
    """assign sys1 node data to sys2 columns"""
    
    att_list = []
    
    blue = sys1_df['sys1 Blue Path'].unique()
    seg_ID = sys1_df['Segment_ID'].unique()
    seg_name = sys1_df['Segment_Name'].unique()
    fam_ID = sys1_df['Family_ID'].unique()
    fam_name = sys1_df['Family_Name'].unique()
    cat_ID = sys1_df['Category_ID'].unique()
    cat_name = sys1_df['Category_Name'].unique()
    
    att_list = sys2_df['sys2_Attribute_Name'].unique()
    
    for att in att_list:
        sys2_df.loc[sys2_df['sys2_Attribute_Name'] == att, 'Category_ID'] = cat_ID
        sys2_df.loc[sys2_df['sys2_Attribute_Name'] == att, 'sys1 Blue Path'] = blue
        sys2_df.loc[sys2_df['sys2_Attribute_Name'] == att, 'Segment_ID'] = seg_ID
        sys2_df.loc[sys2_df['sys2_Attribute_Name'] == att, 'Segment_Name'] = seg_name
        sys2_df.loc[sys2_df['sys2_Attribute_Name'] == att, 'Family_ID'] = fam_ID
        sys2_df.loc[sys2_df['sys2_Attribute_Name'] == att, 'Family_Name'] = fam_name
        sys2_df.loc[sys2_df['sys2_Attribute_Name'] == att, 'Category_Name'] = cat_name
    
    return sys2_df


def sys1_process(sys1_df, sys1_sample, sys1_all, sys2_dict: Dict, k):
    """create a list of sys1 skus, run through through the sys2_skus query and pull sys2 attribute data if skus are present
        concat both dataframs and join them on matching attribute names"""
    
    df = pd.DataFrame()
    order = 'normal'
    count = 1
        
    cat_name = sys1_df['Category_Name'].unique()
    cat_name = list(cat_name)
    cat_name = cat_name.pop()
    print('cat name = {} {}'.format(k, cat_name))
    
    sys1_skus = sys1_df.drop_duplicates(subset='sys1_SKU')  #create list of unique sys1 skus that feed into sys2 query
    sys1_sku_count = len(sys1_skus)
    print('sys1 sku count = ', sys1_sku_count)
    sys1_df = sys1_df.drop_duplicates(subset=['Category_ID', 'sys1_Attr_ID'])  #group by Category_ID and attribute name and keep unique
    sys1_df['sys1 Blue Path'] = sys1_df['Segment_Name'] + ' > ' + sys1_df['Family_Name'] + \
                                                        ' > ' + sys1_df['Category_Name']

    sys1_df = sys1_df.drop(['sys1_SKU', 'sys1_Attribute_Value'], axis=1) #remove unneeded columns
    sys1_df = pd.merge(sys1_df, sys1_sample, on=['sys1_Attribute_Name'])
    sys1_df = pd.merge(sys1_df, sys1_all, on=['sys1_Attr_ID'])
    
    sys1_df['alt_sys1_name'] = process.process_att(sys1_df['sys1_Attribute_Name'])  #prep att name for merge
    #sys1_df.to_csv ("F:/CGabriel/sys1_Shorties/OUTPUT/sys1_test.csv")
    
    sys2_skus = q.sys2_skus(sys1_skus) #get sys2 sku list to determine pim nodes to pull
    if sys2_skus.empty==False:
        #create a dictionary of the unique sys2 nodes that corresponde to the sys1 node
        sys2_l3 = sys2_skus['sys2_Node_ID'].unique()  #create list of pim nodes to pull
        print('sys2 L3s ', sys2_l3)
        
        for node in sys2_l3:
            if node in sys2_dict:
                sys2_df = sys2_dict[node]
            else:
                sys2_dict, sys2_df, order = sys2_process(count, order, node, sys2_dict, k)
            
            if sys2_df.empty==False:
                node_name = sys2_df['sys2_Node_Name'].unique()
                node_name = list(node_name)
                node_name = node_name.pop()
                print('node name = {} {}'.format(node, node_name))
                #add correlating sys1 and sys2 data to opposite dataframes
                sys1_df = sys1_assign_nodes(sys1_df, sys2_df)
                sys2_df = sys2_assign_nodes(sys1_df, sys2_df)
 
                skus = sys2_skus[sys2_skus['sys2_Node_ID'] == node]
                temp_df = pd.merge(sys1_df, sys2_df, left_on=['sys1_Attribute_Name', 'Category_ID', 'sys2_Node_ID', 'sys2_Category_ID', \
                                                                   'sys2_Category_Name', 'sys2_Node_Name', 'sys2_PIM_Path', 'sys1 Blue Path', \
                                                                   'Segment_ID', 'Segment_Name', 'Family_ID', 'Family_Name', 'Category_Name'], 
                                                right_on=['sys2_Attribute_Name', 'Category_ID', 'sys2_Node_ID', 'sys2_Category_ID', \
                                                          'sys2_Category_Name', 'sys2_Node_Name', 'sys2_PIM_Path', 'sys1 Blue Path', \
                                                          'Segment_ID', 'Segment_Name', 'Family_ID', 'Family_Name', 'Category_Name'], how='outer')
                temp_df = match_category(temp_df) #compare sys1 and sys2 atts and create column to say whether they match 
                temp_df['sys1_sku_count'] = sys1_sku_count
                temp_df['sys2_sku_count'] = len(skus)#temp_skus['sys2_SKU']
                temp_df['sys1-sys2 Terminal Node Mapping'] = cat_name+' -- '+node_name
                temp_df['sys2/sys1 SKU Counts'] = temp_df['sys2_sku_count'].map(str)+' / '+temp_df['sys1_sku_count'].map(str)
                
                df = pd.concat([df, temp_df], axis=0, sort=False) #add prepped df for this sys2 node to the final df
            else:
                print('sys2 Node {} EMPTY DATAFRAME'.format(node))
            count += 1

    else:
        print('No sys2 SKUs for sys1 node {}'.format(k))
        
    return df, sys2_dict, order  #where sys2_att_temp is the list of all normalized values for sys2 attributes

In [ ]:
#determine SKU or node search
search_level = 'cat.CATEGORY_ID'

sys2_df = pd.DataFrame()
sys1_df = pd.DataFrame()
sys1_skus = pd.DataFrame()

attribute_df = pd.DataFrame()
sys1_att_vals = pd.DataFrame()
sys1_sample_vals = pd.DataFrame()
sys2_att_vals = pd.DataFrame()
sys2_dict = dict()

column_order = 'normal'
count = 1

data_type = fd.search_type()

if data_type == 'sys1_query':
    search_level = fd.blue_search_level()
    
search_data = fd.data_in(data_type, settings.directory_name)

start_time = time.time()
print('working...')

if data_type == 'sys1_query':
    if search_level == 'cat.CATEGORY_ID':
        for k in search_data:
            sys1_df = q.gcom.sys1_q(sys1_attr_query, search_level, k)
            if sys1_df.empty == False:
                sys1_att_vals, sys1_sample_vals = q.sys1_values(sys1_df)
                sys1_sample_vals = sys1_sample_vals.rename(columns={'sys1_Attribute_Value': 'sys1 Attribute Sample Values'})
                sys1_att_vals = sys1_att_vals.rename(columns={'sys1_Attribute_Value': 'sys1 ALL Values'})
                temp_df, sys2_dict, order = sys1_process(sys1_df, sys1_sample_vals, sys1_att_vals, sys2_dict, k)
                if order == 'alt':
                    column_order = 'alt'
                attribute_df = pd.concat([attribute_df, temp_df], axis=0, sort=False)
                print ('sys1 ', k)
            else:
                print('No attribute data')
    else:
        for k in search_data:
            print('K = ', k)
            sys1_skus = q.sys1_nodes(k, search_level)
            #sys1_skus = pd.concat([sys1_skus, temp_df], axis=0, sort=False)
            sys1_l3 = sys1_skus['Category_ID'].unique()  #create list of pim nodes to pull
            print('sys1 L3s = ', sys1_l3)
            for j in sys1_l3:
                sys1_df = q.gcom.sys1_q(sys1_attr_query, 'cat.CATEGORY_ID', j)
                if sys1_df.empty == False:
                    sys1_att_vals, sys1_sample_vals = q.sys1_values(sys1_df)
                    sys1_sample_vals = sys1_sample_vals.rename(columns={'sys1_Attribute_Value': 'sys1 Attribute Sample Values'})
                    temp_df, sys2_dict, order = sys1_process(sys1_df, sys1_sample_vals, sys1_att_vals, sys2_dict, j)
                    if order == 'alt':
                        column_order = 'alt'
                    attribute_df = pd.concat([attribute_df, temp_df], axis=0, sort=False)
                    print ('sys1 ', j)
                else:
                    print('sys1 node {} All SKUs are R4, R9, or discontinued'.format(j)) 
            print("--- {} seconds ---".format(round(time.time() - start_time, 2)))


attribute_df = attribute_df.drop(['Count'], axis=1)


fd.attribute_match_data_out(settings.directory_name, attribute_df, column_order, search_level)            

print("--- {} seconds ---".format(round(time.time() - start_time, 2)))